# Scraping Yahoo Finance

## Week 7. Practice Programming Assignment 1

In this assignment you are required to look at historical data for 30 companies from [Dow Jones Index](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average). Tickers for companies from the index can be found in *dow_jones_tickers.txt*. For each company you should get historical daily stock prices for 2019 from https://finance.yahoo.com/, and then use the data to answer the questions you will find below. 

### Coding part

In [1]:
from datetime import datetime

from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

In [2]:
with open('dow_jones_tickers.html') as f:
    tickers_soup = BeautifulSoup(f)

tickers_list = [element.text for element in 
                tickers_soup.find_all('td', attrs={'class': "blob-code blob-code-inner js-file-line"})]
tickers_list.sort()
print(tickers_list)

['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'RTX', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT', 'XOM']


In [3]:
url = 'https://finance.yahoo.com/quote/{ticker}/history?period1=1546300800&period2=1577750400&interval=1d&filter=history&frequency=1d'

In [4]:
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Safari/605.1.15'}
# session = Session()
# response = session.get(url.format(ticker='AAPL'), headers=headers)
# response.status_code

In [5]:
# soup = BeautifulSoup(response.text)
# tickers_data = soup.find_all('tr', attrs={'class': "BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)"})
# tickers_data[0]

In [6]:
class at_least_n_present(object):
    def __init__(self, locator, n):
        self.locator = locator
        self.n = n
    
    def __call__(self, locator):
        elements = driver.find_elements(*self.locator)
        if len(elements) >= self.n:
            return elements
        else:
            for _ in range(2):
                driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.PAGE_DOWN)
            return False

In [7]:
def get_ticker_prices(driver, url):
    print('\tDownloading web page...')
    driver.get(url)

    html_rows = WebDriverWait(driver, 8).until(
        at_least_n_present((By.XPATH, "//tr[@class='BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)']"),
                            201)
    )

    rows = [tuple(row.text.split()) for row in html_rows]    
    rows = set(rows)
    print(f'\tGot {len(rows)} unique rows.')
    
    return list(rows)

In [8]:
driver = webdriver.Chrome()

headers = ['month', 'day_of_month', 'year', 'open', 'high', 'low', 'close', 'adj_close', 'volume', 'ticker']
result_df = pd.DataFrame(columns=headers)

for ticker in tickers_list:
    print(f'Start collecting {ticker}.')
    data = get_ticker_prices(driver, url.format(ticker=ticker))
    temp_df = pd.DataFrame(data, columns=headers[:9])
    temp_df['ticker'] = [ticker] * len(data)
    
    result_df = pd.concat([result_df, temp_df])
    temp_df.drop(temp_df.index, inplace=True)
    print(f'Appended {len(data)} rows.')

driver.quit()

result_df.to_csv(f'tickers{datetime.now().strftime('%Y-%m-%dT%H:%M')}.csv', index=False)

Start collecting AAPL.
	Got 255 unique rows.
Appended 255 rows.
Start collecting AXP.
	Got 255 unique rows.
Appended 255 rows.
Start collecting BA.
	Got 255 unique rows.
Appended 255 rows.
Start collecting CAT.
	Got 255 unique rows.
Appended 255 rows.
Start collecting CSCO.
	Got 255 unique rows.
Appended 255 rows.
Start collecting CVX.
	Got 255 unique rows.
Appended 255 rows.
Start collecting DIS.
	Got 253 unique rows.
Appended 253 rows.
Start collecting DOW.
	Got 201 unique rows.
Appended 201 rows.
Start collecting GS.
	Got 255 unique rows.
Appended 255 rows.
Start collecting HD.
	Got 255 unique rows.
Appended 255 rows.
Start collecting IBM.
	Got 255 unique rows.
Appended 255 rows.
Start collecting INTC.
	Got 255 unique rows.
Appended 255 rows.
Start collecting JNJ.
	Got 255 unique rows.
Appended 255 rows.
Start collecting JPM.
	Got 255 unique rows.
Appended 255 rows.
Start collecting KO.
	Got 255 unique rows.
Appended 255 rows.
Start collecting MCD.
	Got 255 unique rows.
Appended 255

In [9]:
tickers_df = result_df.copy()
tickers_df.head()

,month,day_of_month,year,open,high,low,close,adj_close,volume,ticker
0,Jun,"21,",2019,49.70,50.21,49.54,49.69,48.22,"191,202,400",AAPL
1,Jun,"11,",2019,48.72,49.00,48.40,48.70,47.25,"107,731,600",AAPL
2,Mar,"04,",2019,43.92,44.44,43.49,43.96,42.49,"109,744,800",AAPL
3,Feb,"01,",2019,41.74,42.24,41.48,41.63,40.06,"130,672,400",AAPL
4,Nov,"07,",2019,0.1925,Dividend,None,None,None,None,AAPL


In [10]:
tickers_df['date'] = pd.to_datetime(
    tickers_df.day_of_month + tickers_df.month + tickers_df.year,
    format='%d,%b%Y'
)
tickers_df.drop(columns=['month', 'day_of_month', 'year'], inplace=True)

tickers_df['volume'] = tickers_df.volume.str.replace(',', '')

In [19]:
num_cols = tickers_df.columns[:6]
tickers_df[num_cols] = tickers_df[num_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
tickers_df.dropna(inplace=True)

In [42]:
tickers_df = tickers_df.sort_values(by='date').reset_index(drop=True)
tickers_df.head()

,open,high,low,close,adj_close,volume,ticker,date,growth
0,93.91,96.27,93.77,95.68,89.76,4175400.0,AXP,2019-01-02,1.884783
1,42.28,43.20,42.21,42.95,37.45,23833500.0,CSCO,2019-01-02,1.584674
2,91.03,91.39,89.93,91.28,81.45,9843900.0,PG,2019-01-02,0.274635
3,46.94,47.22,46.56,46.93,40.79,11603700.0,KO,2019-01-02,-0.021304
4,316.19,323.95,313.71,323.81,314.65,3292200.0,BA,2019-01-02,2.409943


<br><br>

### Questions

<br><br>

**Question 1.** What is the average change of price over the year (in %)?

*Note 1*. The opening price is the price at which a stock first trades upon the opening of an exchange on a trading day.

*Note 2*. The closing price for any stock is the final price at which it trades during regular market hours on any given day.

*Note 3*. Here by the price change we going to mean a ratio of a closing price in the last day of the period to an opening price in the first day of that period, subtracted one and multiplied by 100.

Example: if a price of a stock in day 1 opened at \\$100, and its close price in the last day was \\$120, then the price change during the period is: $$\left(\dfrac{120}{100}-1\right) * 100 = (1.2 - 1) * 100=20.$$

The price grew by 20%.

In [62]:
 tickers_df['growth'] = (tickers_df.close / tickers_df.open - 1) * 100

In [51]:
open_close_df = tickers_df.groupby('ticker').agg(func={'open': 'first', 'close': 'last'})
open_close_df['growth'] = (open_close_df['close'] / open_close_df['open'] - 1) * 100

In [52]:
answer_part_1 = open_close_df.growth.mean()
print(answer_part_1)

23.285396890298383


<br>

**Question 2.** What company's stock price grew the most (in %)? Enter ticker of the company as an answer)

In [53]:
open_close_df.sort_values(by='growth', ascending=False).head()

,open,close,growth
ticker,,,
AAPL,38.72,72.88,88.223140
MSFT,99.55,157.59,58.302361
V,130.00,187.83,44.484615
JPM,95.95,138.63,44.481501
RTX,66.18,94.32,42.520399


In [54]:
answer_part_2 = 'AAPL'

<br>

**Question 3.** What company's stock lost in price the most (in %)? Enter ticker of the company as an answer

In [55]:
open_close_df.sort_values(by='growth').head()

,open,close,growth
ticker,,,
WBA,67.20,58.91,-12.336310
PFE,40.91,36.92,-9.753117
MMM,187.82,175.83,-6.383772
DOW,52.75,54.34,3.014218
XOM,67.35,69.48,3.162584


In [56]:
answer_part_3 = 'WBA'

<br>

**Question 4.** What company had the largest summary volume over the year? Enter ticker of the company as an answer

In [60]:
tickers_df.groupby('ticker')['volume'].sum().sort_values(ascending=False).head()

ticker
AAPL    2.815414e+10
MSFT    6.176041e+09
PFE     6.038738e+09
INTC    5.505541e+09
CSCO    5.157499e+09
Name: volume, dtype: float64

In [61]:
answer_part_4 = 'AAPL'

<br>

**Question 5.** What is the biggest stock price daily increase (in %)? Enter the number 

In [63]:
tickers_df['growth'].max()

7.740997118000381

In [70]:
answer_part_5 = tickers_df['growth'].max()

<br><br>

**Question 6.** What is the company that had the biggest stock price daily increase? Enter ticker of the company as an answer

In [71]:
tickers_df.query('growth == @answer_part_5')

,open,high,low,close,adj_close,volume,ticker,date,growth
1340,371.27,402.67,365.55,400.01,390.64,34742200.0,BA,2019-03-11,7.740997


In [ ]:
answer_part_6 = 'BA'

<br>

**Question 7.** What is the biggest stock price daily decrease (in %)? Enter the number

In [72]:
tickers_df['growth'].min()

-7.16356455611108

In [74]:
answer_part_7 = tickers_df['growth'].min()

<br>

**Question 8.** What is the company that had the biggest stock price daily decrese (in %)? Enter ticker of the company as an answer

In [75]:
tickers_df.query('growth == @answer_part_7')

,open,high,low,close,adj_close,volume,ticker,date,growth
2108,238.01,238.01,215.82,220.96,207.34,27361400.0,UNH,2019-04-16,-7.163565


In [ ]:
answer_part_8 = 'UNH'

<br>

**Question 9.** What was the best month for all companies (i.e. average monthly price increase was the best)? Enter one of the following: January, February, March, April, May, June, July, August, September, October, November, December

In [79]:
tickers_df['month'] = tickers_df.date.dt.month
open_close_month = tickers_df.pivot_table(index=['ticker', 'month'],
                                          values=['open', 'close'],
                                          aggfunc={'open': 'first', 'close': 'last'}
                                         )
open_close_month['growth'] = (open_close_month.close / open_close_month.open - 1) * 100

In [84]:
open_close_month.groupby('month')['growth'].mean().sort_values(ascending=False)

month
1     7.409273
6     7.062817
9     3.443022
2     3.382967
11    2.573628
4     1.984414
12    1.802325
10    0.697505
3     0.303609
7    -0.240822
8    -2.751223
5    -6.664097
Name: growth, dtype: float64

In [ ]:
answer_part_9 = 'January'

<br>

**Question 10.** What was the worst month for all companies (i.e. average monthly price increase was the worst)? Enter one of the following: January, February, March, April, May, June, July, August, September, October, November, December

In [85]:
answer_part_10 = 'May'

<br>
<br>

#### Submit your answers